In [48]:
import pandas as pd
import sqlite3
import yfinance as yf
import requests
import json

In [49]:
start_date="2018-01-01"
end_date="2023-12-31"
tickers=["AAPL","MSFT",'SPY']
data=pd.DataFrame()
conection=sqlite3.connect('database.db')

In [50]:
def download_rename(data, name):
    try:
        del data['t']
    except:
        pass
    data.rename(columns={'o':f'{name}_Open'},inplace=True)
    data.rename(columns={'h':f'{name}_High'},inplace=True)
    data.rename(columns={'l':f'{name}_Low'},inplace=True)
    data.rename(columns={'c':f'{name}_Close'},inplace=True)
    data.rename(columns={'a':f'{name}_Adj Close'},inplace=True)
    data.rename(columns={'v':f'{name}_Volume'},inplace=True)
    return data

In [51]:
api_key='dd75440b5984491b9f3593d8cc275ed4'
headers = {"Authorization": f"Bearer {api_key}"}
for i in range(len(tickers)):
    #hystorical_daily_request=f'https://api.darqube.com/data-api/market-data/historical/daily/TSLA?token={api_key}&start_date={start_date}&end_date={end_date}'
    try:
        response=requests.get(f'https://api.darqube.com/data-api/market-data/historical/daily/{tickers[i]}?token={api_key}&start_date={start_date}&end_date={end_date}')
        response.raise_for_status()
        data_json=response.json()
        if data.empty:
            data=download_rename(pd.DataFrame(data_json),tickers[i])
        else:
            data=pd.merge(data,download_rename(pd.DataFrame(data_json),tickers[i]),left_index=True,right_index=True)
    except requests.exceptions.RequestException as e:
        print(f"API Error: {e}")



In [53]:
data.to_sql('stocks',conection,index=False,if_exists='replace')

1510